In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import email as em

from datetime import date, timedelta, datetime
from time import mktime
from email.utils import parsedate, parsedate_tz, mktime_tz
from email.parser import HeaderParser

from classes.gmail import GmailAccount

In [3]:
password = open(os.path.expanduser('~/.ssh/pw'), 'r').read().strip()

In [5]:
tyler = GmailAccount(username='tyleha@gmail.com', password=password)
tyler.login()

('OK', [b'tyleha@gmail.com authenticated (Success)'])

In [6]:
daysback = 70
notsince = 0
since = (date.today() - timedelta(daysback)).strftime("%d-%b-%Y")
before = (date.today() - timedelta(notsince)).strftime("%d-%b-%Y")

SEARCH = '(SENTSINCE {si} SENTBEFORE {bf})'.format(si=since, bf=before)
BODY = '(BODY.PEEK[TEXT])'
ALL_HEADERS = '(BODY.PEEK[HEADER.FIELDS (DATE TO CC FROM SUBJECT)])'
DATE = '(BODY.PEEK[HEADER.FIELDS (DATE)])'

In [12]:
#LOAD GMAIL EMAILS
received = tyler.load_parse_query(SEARCH, ALL_HEADERS, '"[Gmail]/All Mail"')
# sent = tyler.load_parse_query(SEARCH, ALL_HEADERS, '"[Gmail]/Sent Mail"')

In [8]:
def load_pickle(filepath, ftype='r'):
    """opens and closes pickled file and returns contained pickleobj"""
    f = open(filepath, ftype)
    contents = pickle.load(f)
    f.close()
    return contents    

def save_pickle(content, filepath, ftype='w'):
    """
    :return: pickle's weird dump output. not useful
    :content: the object to be saved (can be any type)
    :filepath: the string path, relative or not, at which to save
    :ftype: 'w' to write, 'wb' to write binary
    """
    f = open(filepath, ftype)
    xx = pickle.dump(content, f)
    f.close()
    return xx

# Do the pandas thing

In [9]:
import pandas as pd
import numpy as np
import pickle

In [10]:
def scrub_email(headers):    
    d = {}
    for val in headers:
        d[val[0].lower()] = val[1]
    return d

def try_parse_date(d):
    try:
        ts = pd.Timestamp(d)

        # IMAP is very much not perfect...some of my emails have no timezone
        # in their date string. ¯\_(ツ)_/¯
        if ts.tz is None: 
            ts = ts.tz_localize('UTC')

        # I moved from east coast to west coast in fall 2010, so automatically assume EST/PST 
        # before/after that date.
        if ts < pd.Timestamp('2010-09-04', tz='US/Eastern'):
            ts = ts.tz_convert('US/Eastern')
        else:
            ts = ts.tz_convert('US/Pacific')
        # I would have liked to avoid this whole parse_date method altogether and instead just
        # use pd.Timestamp(df.date), however Pandas is _really_ not built to handle DatetimeIndex
        # or PeriodIndex of anything other than a single timezone (see http://stackoverflow.com/a/17027507/1766755)
        # Because we care about timezone-naive datestamps and Pandas forces us to be timezone aware, 
        # we have to fall back to the datetime library and strip timezone from all our dates
        # using tzinfo=None.
        return pd.Timestamp(ts.to_datetime().replace(tzinfo=None))
    
    except:
        # Sometimes, IMAP just has dates formatted in a totally improper 
        # way...see 'Thursday , 10 Dec 2009 16:28:55, PST'. These are so few and far between and so
        # irregular that we'll just remove them from our dataset.
        return np.nan

In [6]:
received = load_pickle('/home/tyleha/git/all_emails', 'rb')

In [13]:
freq = 'M'
email_dataset = received
df = pd.DataFrame([scrub_email(email._headers) for email in email_dataset])

In [14]:
# Frequency
freq = 'M'
email_dataset = sent
df = pd.DataFrame([scrub_email(email._headers) for email in email_dataset])
 
# This might take a minute...
df['timestamp'] = df.date.map(try_parse_date)
df = df.dropna(subset=['timestamp'])


In [16]:
# Now we're off to the races
df['hour'] = df.timestamp.map(lambda x: x.hour)
df = df.set_index('timestamp', drop=False)
df.index = df.index.to_period(freq)


In [35]:
# Scale our heatmap to the min/max range of our email data.
mindate = df.timestamp.min()
maxdate = df.timestamp.max()
pr = pd.period_range(mindate, maxdate, freq=freq)

# Initialize a new HeatMap dataframe where the indicies are actually Periods of time!
# Size the frame anticipating the correct number of rows (periods) and columns (hours in a day)
hm = pd.DataFrame(np.zeros([len(pr), 24]) , index=pr)

from classes.progress import ProgressBar

prog = ProgressBar(len(pr))
for i, period in enumerate(pr):
    # HERE'S where the magic happens...with pandas, when you structure your data correctly, it can be so terse that
    # you almost aren't sure the program does what it says it does...
    # For this period (month), find all emails within this month and count how many emails were received
    # within each hour of the day in that month. Wow. Takes more words to explain than to code.
    # Normally that would take you 3 or 4 for loops, constantly trying to catch edge cases and fencepost
    # problems. But here, it's as simple as you please.
    if period in df.index:
        hm.ix[period] = df.loc[[period]].hour.value_counts()
    prog.animate(i)
    
# If for some weird reason there was ever an hour period where you had no email,
# fill those NaNs with zeros.
hm.fillna(0, inplace=True)

 4 of 4    [##################### 100% #####################]    Time: 0:00:00    ETA: ...

# Heatmap Plot

In [ ]:
# Hide that time I sent a bajillion wedding save the date emails
hm.ix[pd.Period('2013-04', 'M')][18] = 20

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as dates
from datetime import time
import matplotlib.gridspec as gridspec
%matplotlib inline

In [ ]:
# Set up figure
fig = plt.figure(figsize=(12,8))
gs = gridspec.GridSpec(2, 2, height_ratios=[4,1], width_ratios=[20,1],)
gs.update(wspace=0.05)

### Plot our heatmap
ax = plt.subplot(gs[0])
x = dates.date2num([p.start_time for p in pr])
t = [datetime(2000, 1, 1, h, 0, 0) for h in range(24)]
t.append(datetime(2000, 1, 2, 0, 0, 0))
y = dates.date2num(t)
cm = plt.get_cmap('Blues')
plt.pcolor(x, y, hm.transpose().as_matrix(), cmap=cm)

date_format = dates.DateFormatter('%b %Y')
ax.xaxis.set_major_formatter(date_format)
ax.yaxis.set_major_formatter(dates.DateFormatter('%H:%M'))
ax.set_yticks(t[::2])
ax.set_xticks(x[::12])
# fig.autofmt_xdate()

ax.set_xlim([x[0], x[-1]])
ax.set_ylim([t[0], t[-1]])

# Add a colorbar!
plt.colorbar(cax=plt.subplot(gs[1]))

#### Plot total emails over time 
ax2 = plt.subplot(gs[2])
g = df.groupby(level=0)
total_email = g.hour.count()
plt.plot_date(total_email.index, total_email, '-', linewidth=1.5, color=cm(0.999))
ax2.get_xaxis().set_visible(False)
ax2.get_yaxis().set_visible(False)

# Scratch City

In [ ]:
pd.crosstab()

In [ ]:
ax = plt.imshow(hm.transpose(), interpolation='nearest', cmap='Oranges').axes

In [ ]:
x = parsedate_tz('Fri, 3 Sep 2010 21:26:05 -0500')
x = parsedate_tz('Thu, 31 Dec 2015 02:07:11 +0000')
x

In [ ]:
y = pd.Timestamp('Thu, 31 Dec 2015 02:07:11 +0000')
y = y.tz_convert('US/Eastern')
pd.Timestamp(y.to_datetime().replace(tzinfo=None)).tz_localize('UTC')

In [ ]:
pd.Timestamp('Fri, 3 Sep 2010 21:26:05 -0500').to_datetime().replace(tzinfo=None)